<a href="https://colab.research.google.com/github/deni1171/deni-resolusi/blob/master/nlpdicodingtraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ekstraksi file zip
import zipfile
import os
local_zip = '/content/drive/My Drive/datasets/Text Emotion.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [4]:
import pandas as pd
df = pd.read_csv('/tmp/Text Emotion/text_emotion.csv')
df = df.drop(columns=['tweet_id', 'author'])
df['content'] = df['content'].str.lower()
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,layin n bed with a headache ughhhh...waitin o...
2,sadness,funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends soon!
4,neutral,@dannycastillo we want to trade with someone w...


In [5]:
# membuat one-hot-encoding dan dataset baru
category = pd.get_dummies(df.sentiment)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='sentiment')
df_baru

,content,anger,boredom,empty,enthusiasm,fun,happiness,hate,love,neutral,relief,sadness,surprise,worry
0,@tiffanylue i know i was listenin to bad habi...,0,0,1,0,0,0,0,0,0,0,0,0,0
1,layin n bed with a headache ughhhh...waitin o...,0,0,0,0,0,0,0,0,0,0,1,0,0
2,funeral ceremony...gloomy friday...,0,0,0,0,0,0,0,0,0,0,1,0,0
3,wants to hang out with friends soon!,0,0,0,1,0,0,0,0,0,0,0,0,0
4,@dannycastillo we want to trade with someone w...,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,@johnlloydtaylor,0,0,0,0,0,0,0,0,1,0,0,0,0
39996,happy mothers day all my love,0,0,0,0,0,0,0,1,0,0,0,0,0
39997,happy mother's day to all the mommies out ther...,0,0,0,0,0,0,0,1,0,0,0,0,0
39998,@niariley wassup beautiful!!! follow me!! pee...,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
inggris = stopwords.words('english')
corpus = []
for i in range(0, len(df_baru)):
  content = re.sub('[^a-zA-Z]', ' ', df_baru['content'][i])
  content = content.lower()
  content = content.split()
  ps = PorterStemmer()
  content = [ps.stem(word) for word in content if not word in inggris]
  content = ' '.join(content)
  corpus.append(content)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# dataframe ke numpy array
sentiment = df_baru['content'].values
label = df_baru[['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness', 'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise', 'worry']]

In [8]:
# membagi data train dan test
from sklearn.model_selection import train_test_split
sentiment_train, sentiment_test, label_train, label_test = train_test_split(sentiment, label, test_size=0.2)

In [9]:
# ubah kata menjadi numerik
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sentiment_train)

sequence_train = tokenizer.texts_to_sequences(sentiment_train)
sequence_test = tokenizer.texts_to_sequences(sentiment_test)

padded_train = pad_sequences(sequence_train)
padded_test = pad_sequences(sequence_test)

In [10]:
# memperpendek waktu pelatihan
accuracy = 0.95

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= accuracy):
      print('mencapai akurasi yg diinginkan')
      self.model.stop_training=True

callbacks = myCallback

In [11]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_dim=9000, output_dim=16),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(13, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
# training model
num_epochs = 100
history = model.fit(padded_train, label_train, epochs=num_epochs, validation_data=(padded_test, label_test), verbose=2, callbacks = [callbacks()])

Epoch 1/100
1000/1000 - 19s - loss: 2.0607 - accuracy: 0.2805 - val_loss: 1.9680 - val_accuracy: 0.3191
Epoch 2/100
1000/1000 - 18s - loss: 1.8724 - accuracy: 0.3589 - val_loss: 1.9214 - val_accuracy: 0.3409
Epoch 3/100
1000/1000 - 18s - loss: 1.7930 - accuracy: 0.3880 - val_loss: 1.9257 - val_accuracy: 0.3490
Epoch 4/100
1000/1000 - 18s - loss: 1.7366 - accuracy: 0.4101 - val_loss: 1.9660 - val_accuracy: 0.3483
Epoch 5/100
1000/1000 - 19s - loss: 1.6848 - accuracy: 0.4286 - val_loss: 1.9643 - val_accuracy: 0.3424
Epoch 6/100
1000/1000 - 18s - loss: 1.6320 - accuracy: 0.4486 - val_loss: 2.0118 - val_accuracy: 0.3391
Epoch 7/100
1000/1000 - 18s - loss: 1.5847 - accuracy: 0.4635 - val_loss: 2.0768 - val_accuracy: 0.3286
Epoch 8/100
1000/1000 - 19s - loss: 1.5366 - accuracy: 0.4806 - val_loss: 2.1514 - val_accuracy: 0.3385
Epoch 9/100
1000/1000 - 18s - loss: 1.4853 - accuracy: 0.4960 - val_loss: 2.2403 - val_accuracy: 0.3295
Epoch 10/100
1000/1000 - 18s - loss: 1.4369 - accuracy: 0.5120 -

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
# grafik plot loss accuracy training model
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation Accuracy Scores')

plt.legend()

plt.show()